In [1]:
from glob import glob
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from colorbar import ColorbarTemplate
from tqdm.auto import tqdm
import multiprocessing


In [4]:
#! mkdir output/images_preprocessed_bkp

In [6]:
#! cp output/images_preprocessed/* output/images_preprocessed_bkp

/bin/bash: /usr/bin/cp: Argument list too long


In [8]:
colorbar = ColorbarTemplate()
colorbar_short = ColorbarTemplate(fn='data/colorbar_short.png')
colorbar_vertical_short = ColorbarTemplate(fn='data/colorbar_vertical_short.png')
colorbar_short_flat = ColorbarTemplate(fn='data/colorbar_short_flat.png')
colorbar_colorbar = ColorbarTemplate(fn='data/colorbar.png')
colorbar_colorbar_2 = ColorbarTemplate(fn='data/colorbar_2.png')
colorbar_colorbar_3 = ColorbarTemplate(fn='data/colorbar_3.png')
colorbar_light = ColorbarTemplate(fn='data/colorbar_light.png')
colorbar_long = ColorbarTemplate(fn='data/colorbar_long.png')
colorbar_ruler = ColorbarTemplate(fn='data/colorbar_ruler.png')
colorbar_1cm = ColorbarTemplate(fn='data/colorbar_1cm.png')
colorbar_2cm = ColorbarTemplate(fn='data/colorbar_2cm.png')
colorbar_none2 = ColorbarTemplate(fn='data/colorbar_none2.png')
colorbar_none = ColorbarTemplate(fn='data/colorbar_none.png')
colorbar_grey = ColorbarTemplate(fn='data/colorbar_grey.png')

In [9]:
def process_fn(fn):
    out_fn = fn.replace('/images/', '/images_preprocessed/')
    if not os.path.exists(out_fn):
        img = np.asarray(Image.open(fn))
        out_img = colorbar.erase_from_image(img)
        out_img = colorbar_short.erase_from_image(out_img)
        out_img = colorbar_vertical_short.erase_from_image(out_img)
        out_img = colorbar_short_flat.erase_from_image(out_img)
        out_img = colorbar_colorbar.erase_from_image(out_img)
        out_img = colorbar_colorbar_2.erase_from_image(out_img)
        out_img = colorbar_colorbar_3.erase_from_image(out_img)
        out_img = colorbar_light.erase_from_image(out_img)
        out_img = colorbar_long.erase_from_image(out_img)
        out_img = colorbar_ruler.erase_from_image(out_img)
        out_img = colorbar_1cm.erase_from_image(out_img)
        out_img = colorbar_2cm.erase_from_image(out_img)
        out_img = colorbar_none2.erase_from_image(out_img)
        out_img = colorbar_none.erase_from_image(out_img)
        out_img = colorbar_grey.erase_from_image(out_img)
        out_img_pil = Image.fromarray(out_img)
        out_img_pil.save(out_fn)

In [10]:
FNS = glob('output/images/*.png')

In [11]:
with multiprocessing.Pool(16) as p:
    it = p.imap(process_fn, FNS)
    for _ in tqdm(it, total=len(FNS)):
        ...

  0%|          | 0/97640 [00:00<?, ?it/s]